In [1]:
# Standard Imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
# Additional Imports
import os, json, math, time
from yelpapi import YelpAPI
from tqdm.notebook import tqdm_notebook

In [2]:
# Load API Credentials
with open('/Users/christianrim/.secret/yelp_api.json') as f:   #use your path here!
    login = json.load(f)

In [3]:
# Instantiate YelpAPI Variable
yelp_api = YelpAPI(login['api-key'], timeout_s=5.0)
yelp_api

In [5]:
# Quick Test Query
results = yelp_api.search_query(location='Los Angeles, CA',
                                       term='Korean')
print(type(results))
results.keys()

<class 'dict'>


dict_keys(['businesses', 'total', 'region'])

In [6]:
# set our API call parameters 
LOCATION = '3744 W 6th St'
TERM = 'Korean'

In [7]:
# create a JSON_FILE filename
# include the search terms 
JSON_FILE = "Data/results_in_progress_korean_food.json"
JSON_FILE

'Data/results_in_progress_korean_food.json'

In [8]:
## checking if the file is there 
file_exists = os.path.isfile(JSON_FILE)
## If it does not exist: 
if file_exists == False:
    
    ## CREATE ANY NEEDED FOLDERS
    # Get the Folder Name only
    folder = os.path.dirname(JSON_FILE)
    ## If JSON_FILE included a folder:
    if len(folder)>0:
        # create the folder
        os.makedirs(folder,exist_ok=True)
        
        
    ## INFORM USER AND SAVE EMPTY LIST
    print(f'[i] {JSON_FILE} not found. Saving empty list to file.')
    
    
    # save an empty list
    with open(JSON_FILE,'w') as f:
        json.dump([],f)  
# If it exists, inform user
else:
    print(f"[i] {JSON_FILE} already exists.")

[i] Data/results_in_progress_korean_food.json already exists.


Determines how many results are already in the file

In [9]:
## Load previous results and use len of results for offset
with open(JSON_FILE,'r') as f:
    previous_results = json.load(f)
    
## set offset based on previous results
n_results = len(previous_results)
print(f'- {n_results} previous results found.')

- 200 previous results found.


Figure out how many pages of results we will need 

In [10]:
# use our yelp_api variable's search_query
results = yelp_api.search_query(location=LOCATION,
                               term=TERM,
                               offset=n_results)
results.keys()

dict_keys(['businesses', 'total', 'region'])

In [11]:
## how many results total?
total_results = results['total']
total_results

975

In [12]:
# how many did we get the details for? 
results_per_page = len(results['businesses'])
results_per_page

20

From our queries, there are 969 businesses to retrieve from our API and we get 20 results at a time. 
- Can calculate the number of results remaining by subtracting our offset from our total
- can determine how many pages we will need to divide the results by 20


In [13]:
import time, math
# use math.cell to round up for the total number of pages of results
n_pages = math.ceil((results['total']-n_results)/ results_per_page)
n_pages

39

In [14]:
# join new results with old list with extend and save to file
previous_results.extend(results['businesses'])  
with open(JSON_FILE,'w') as f:
     json.dump(previous_results,f)

In [15]:
from tqdm.notebook import tqdm_notebook
import time
for i in tqdm_notebook(range(n_pages)):
    # adds 200 ms pause
    time.sleep(.2) 

  0%|          | 0/39 [00:00<?, ?it/s]

In [16]:
for i in tqdm_notebook( range(1,n_pages+1)):
    
    ## Read in results in progress file and check the length
    with open(JSON_FILE, 'r') as f:
        previous_results = json.load(f)
    ## save number of results for to use as offset
    n_results = len(previous_results)
    ## use n_results as the OFFSET 
    results = yelp_api.search_query(location=LOCATION,
                                    term=TERM, 
                                    offset=n_results)
    
    ## append new results and save to file
    previous_results.extend(results['businesses'])
    
    with open(JSON_FILE,'w') as f:
        json.dump(previous_results,f)
    
    # add a 200ms pause
    time.sleep(.2)

  0%|          | 0/39 [00:00<?, ?it/s]

## Converting JSON to dataframe

In [17]:
# load final results
final_df = pd.read_json(JSON_FILE)
display(final_df.head(), final_df.tail())

,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,price,location,phone,display_phone,distance
0,h1R2iKYdm2lwukzMJvJqDw,hangari-kalguksu-los-angeles-4,Hangari Kalguksu,https://s3-media3.fl.yelpcdn.com/bphoto/X_U65O...,False,https://www.yelp.com/biz/hangari-kalguksu-los-...,2385,"[{'alias': 'korean', 'title': 'Korean'}, {'ali...",4.5,"{'latitude': 34.0628602582049, 'longitude': -1...","[pickup, delivery]",$$,"{'address1': '3470 W 6th St', 'address2': 'Ste...",+12133882326,(213) 388-2326,584.192122
1,WO12S4gvOYkYF8_70RWrQA,haneuem-los-angeles-3,HanEuem,https://s3-media3.fl.yelpcdn.com/bphoto/Rh_Q54...,False,https://www.yelp.com/biz/haneuem-los-angeles-3...,190,"[{'alias': 'korean', 'title': 'Korean'}, {'ali...",4.5,"{'latitude': 34.06427634725579, 'longitude': -...","[delivery, pickup]",NaN,"{'address1': '539 S Western Ave', 'address2': ...",+12133888988,(213) 388-8988,527.290392
2,uzAbw27XQTXTivjgf2bN2w,han-bat-sul-lung-tang-los-angeles-2,Han Bat Sul Lung Tang,https://s3-media2.fl.yelpcdn.com/bphoto/YttPox...,False,https://www.yelp.com/biz/han-bat-sul-lung-tang...,2666,"[{'alias': 'korean', 'title': 'Korean'}, {'ali...",4.5,"{'latitude': 34.065414, 'longitude': -118.3095...",[delivery],$$,"{'address1': '4163 W 5th St', 'address2': '', ...",+12133839499,(213) 383-9499,568.284344
3,qAAoilHU25Qr45FOQlA19g,kang-ho-dong-baekjeong-los-angeles-4,Kang Ho-dong Baekjeong,https://s3-media1.fl.yelpcdn.com/bphoto/0LQSEL...,False,https://www.yelp.com/biz/kang-ho-dong-baekjeon...,4902,"[{'alias': 'bbq', 'title': 'Barbeque'}, {'alia...",4.5,"{'latitude': 34.06375298900917, 'longitude': -...","[delivery, pickup]",$$$,"{'address1': '3465 W 6th St', 'address2': 'Ste...",+12133849678,(213) 384-9678,607.771480
4,iRjnWYY5oVJTaZ5teU1vOQ,king-chang-la-los-angeles,King Chang - LA,https://s3-media3.fl.yelpcdn.com/bphoto/FGJWXO...,False,https://www.yelp.com/biz/king-chang-la-los-ang...,283,"[{'alias': 'korean', 'title': 'Korean'}, {'ali...",5.0,"{'latitude': 34.06329, 'longitude': -118.30372}","[delivery, pickup, restaurant_reservation]",NaN,"{'address1': '3732 W 6th St', 'address2': None...",+12133185250,(213) 318-5250,14.161470


,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,price,location,phone,display_phone,distance
970,AFdPBnkFNLAzsH6WcPuBkw,sushi-and-teri-los-angeles,Sushi & Teri,https://s3-media2.fl.yelpcdn.com/bphoto/oZgYb6...,False,https://www.yelp.com/biz/sushi-and-teri-los-an...,387,"[{'alias': 'sushi', 'title': 'Sushi Bars'}, {'...",3.0,"{'latitude': 34.0490383, 'longitude': -118.239...","[delivery, pickup]",$,"{'address1': '116 Japanese Village Plz Mall', ...",+12136878368,(213) 687-8368,6112.604442
971,H3kFTJbXNuJohjQeELSz3w,teru-sushi-studio-city-2,Teru Sushi,https://s3-media4.fl.yelpcdn.com/bphoto/CzYuBk...,False,https://www.yelp.com/biz/teru-sushi-studio-cit...,900,"[{'alias': 'sushi', 'title': 'Sushi Bars'}, {'...",4.0,"{'latitude': 34.14299, 'longitude': -118.392181}","[delivery, pickup]",$$,"{'address1': '11940 Ventura Blvd', 'address2':...",+18187636201,(818) 763-6201,12013.394326
972,Elt1_91j87F4ePzGPWZJ9Q,thai-original-bbq-glendale,Thai Original BBQ,https://s3-media3.fl.yelpcdn.com/bphoto/u8SQzN...,False,https://www.yelp.com/biz/thai-original-bbq-gle...,603,"[{'alias': 'thai', 'title': 'Thai'}, {'alias':...",3.0,"{'latitude': 34.144812, 'longitude': -118.248078}","[delivery, pickup]",$$,"{'address1': '143 S Glendale Ave', 'address2':...",+18185076789,(818) 507-6789,10403.018146
973,g1K4Nr5zrIYxWu57Uq3bog,california-roll-and-sushi-culver-city,California Roll & Sushi,https://s3-media1.fl.yelpcdn.com/bphoto/NBgY6_...,False,https://www.yelp.com/biz/california-roll-and-s...,165,"[{'alias': 'sushi', 'title': 'Sushi Bars'}, {'...",3.0,"{'latitude': 33.982834269309, 'longitude': -11...","[delivery, pickup]",$$,"{'address1': '6251 Bristol Pkwy', 'address2': ...",+13103377775,(310) 337-7775,12123.651564
974,S8A4xHxDFiklFowZzukKlw,ono-hawaiian-bbq-culver-city-2,Ono Hawaiian BBQ,https://s3-media1.fl.yelpcdn.com/bphoto/MX70jp...,False,https://www.yelp.com/biz/ono-hawaiian-bbq-culv...,94,"[{'alias': 'hawaiian', 'title': 'Hawaiian'}, {...",3.0,"{'latitude': 33.985973, 'longitude': -118.3932}","[delivery, pickup]",$,"{'address1': '6000 Sepulveda Blvd', 'address2'...",+13103906888,(310) 390-6888,11916.550669


In [18]:
# sorting the values by 
final_df.sort_values(by=['rating'], ascending=False)

,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,price,location,phone,display_phone,distance
723,E-nuI1j4NTbWH4PDgCpWeA,roller-rice-los-angeles,Roller Rice,https://s3-media3.fl.yelpcdn.com/bphoto/lp0Y84...,False,https://www.yelp.com/biz/roller-rice-los-angel...,1,"[{'alias': 'korean', 'title': 'Korean'}]",5.0,"{'latitude': 34.0373001, 'longitude': -118.252...",[],NaN,"{'address1': '517 E Olympic Blvd', 'address2':...",+12136238866,(213) 623-8866,5560.749187
673,by1MU_kylxTepZa-XF51mg,cafe-beverly-los-angeles,Cafe Beverly,,False,https://www.yelp.com/biz/cafe-beverly-los-ange...,1,"[{'alias': 'karaoke', 'title': 'Karaoke'}]",5.0,"{'latitude': 34.0576949715614, 'longitude': -1...",[],$$$,"{'address1': '3170 W 8th St', 'address2': '', ...",+12133650345,(213) 365-0345,935.275087
173,CcqraT0cuGKYEcZ1ri_kxg,broken-mouth-lees-homestyle-los-angeles-5,BROKEN MOUTH | Lee's Homestyle,https://s3-media1.fl.yelpcdn.com/bphoto/IiLJzH...,False,https://www.yelp.com/biz/broken-mouth-lees-hom...,1479,"[{'alias': 'hawaiian', 'title': 'Hawaiian'}, {...",5.0,"{'latitude': 34.042734, 'longitude': -118.250779}","[delivery, pickup]",$$,"{'address1': '718 S Los Angeles St', 'address2...",+12134189588,(213) 418-9588,5417.415420
399,ahjbni41lzR7LKiXGcuevQ,ellie-julie-los-angeles-3,Ellie Julie,https://s3-media2.fl.yelpcdn.com/bphoto/aRsdO5...,False,https://www.yelp.com/biz/ellie-julie-los-angel...,22,"[{'alias': 'chickenshop', 'title': 'Chicken Sh...",5.0,"{'latitude': 34.0551603, 'longitude': -118.308...",[],NaN,"{'address1': '928 S Western Ave', 'address2': ...",+12139087293,(213) 908-7293,1011.984289
81,T3cqmVnThp8qA0JV7Z2D4w,hanu-korean-bbq-los-angeles-2,Hanu Korean BBQ,https://s3-media1.fl.yelpcdn.com/bphoto/ZbZwNv...,False,https://www.yelp.com/biz/hanu-korean-bbq-los-a...,789,"[{'alias': 'korean', 'title': 'Korean'}, {'ali...",5.0,"{'latitude': 34.06386, 'longitude': -118.28675}",[],NaN,"{'address1': '2999 W 6th St', 'address2': 'Ste...",+12137387370,(213) 738-7370,1574.983670
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
953,wR43nJ6MLReCdNofwetZyw,crispy-bird-studio-city,Crispy Bird,,False,https://www.yelp.com/biz/crispy-bird-studio-ci...,1,"[{'alias': 'fooddeliveryservices', 'title': 'F...",1.0,"{'latitude': 34.140284, 'longitude': -118.375594}","[delivery, pickup]",NaN,"{'address1': '11288 Ventura Blvd', 'address2':...",+18185794099,(818) 579-4099,10799.249267
547,qEiwIfjt-_X7kyGODtPW7Q,hans-bibimbap-los-angeles-5,Han's Bibimbap,,False,https://www.yelp.com/biz/hans-bibimbap-los-ang...,1,"[{'alias': 'korean', 'title': 'Korean'}]",1.0,"{'latitude': 34.060738, 'longitude': -118.306287}",[],$,"{'address1': '3680 Wilshire Blvd', 'address2':...",+12133848989,(213) 384-8989,375.422046
883,BVnMCBAXWPCBmeFxXbN0kQ,wings-out-los-angeles-2,Wings Out,https://s3-media4.fl.yelpcdn.com/bphoto/tJb4tZ...,False,https://www.yelp.com/biz/wings-out-los-angeles...,2,"[{'alias': 'fooddeliveryservices', 'title': 'F...",1.0,"{'latitude': 33.97845, 'longitude': -118.3926}","[delivery, pickup]",NaN,"{'address1': '6081 Center Drive', 'address2': ...",,,12513.034737
600,r_QpBn5CwnlYHENPCoRpnA,na-go-ya-los-angeles,Na Go Ya,,False,https://www.yelp.com/biz/na-go-ya-los-angeles?...,1,"[{'alias': 'korean', 'title': 'Korean'}]",1.0,"{'latitude': 34.0561999, 'longitude': -118.30941}",[],$$,"{'address1': '869 S Western Ave', 'address2': ...",,,968.416529


# Data Cleaning

In [18]:
# check for duplicate results
final_df.duplicated(subset='id').sum()

0

In [19]:
## Drop duplicate ids and confirm there are no more duplicates
final_df = final_df.drop_duplicates(subset='id')
final_df.duplicated(subset='id').sum()

0

In [20]:
# save the final results to a compressed csv
final_df.to_csv('Data/final_results_LA_korean.csv.gz', compression='gzip',index=False)

In [21]:
final_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 200 entries, 0 to 199
Data columns (total 16 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   id             200 non-null    object 
 1   alias          200 non-null    object 
 2   name           200 non-null    object 
 3   image_url      200 non-null    object 
 4   is_closed      200 non-null    bool   
 5   url            200 non-null    object 
 6   review_count   200 non-null    int64  
 7   categories     200 non-null    object 
 8   rating         200 non-null    float64
 9   coordinates    200 non-null    object 
 10  transactions   200 non-null    object 
 11  price          164 non-null    object 
 12  location       200 non-null    object 
 13  phone          200 non-null    object 
 14  display_phone  200 non-null    object 
 15  distance       200 non-null    float64
dtypes: bool(1), float64(2), int64(1), object(12)
memory usage: 25.2+ KB


In [22]:
final_df.head()

,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,price,location,phone,display_phone,distance
0,h1R2iKYdm2lwukzMJvJqDw,hangari-kalguksu-los-angeles-4,Hangari Kalguksu,https://s3-media3.fl.yelpcdn.com/bphoto/X_U65O...,False,https://www.yelp.com/biz/hangari-kalguksu-los-...,2385,"[{'alias': 'korean', 'title': 'Korean'}, {'ali...",4.5,"{'latitude': 34.0628602582049, 'longitude': -1...","[pickup, delivery]",$$,"{'address1': '3470 W 6th St', 'address2': 'Ste...",+12133882326,(213) 388-2326,584.192122
1,WO12S4gvOYkYF8_70RWrQA,haneuem-los-angeles-3,HanEuem,https://s3-media3.fl.yelpcdn.com/bphoto/Rh_Q54...,False,https://www.yelp.com/biz/haneuem-los-angeles-3...,190,"[{'alias': 'korean', 'title': 'Korean'}, {'ali...",4.5,"{'latitude': 34.06427634725579, 'longitude': -...","[delivery, pickup]",NaN,"{'address1': '539 S Western Ave', 'address2': ...",+12133888988,(213) 388-8988,527.290392
2,uzAbw27XQTXTivjgf2bN2w,han-bat-sul-lung-tang-los-angeles-2,Han Bat Sul Lung Tang,https://s3-media2.fl.yelpcdn.com/bphoto/YttPox...,False,https://www.yelp.com/biz/han-bat-sul-lung-tang...,2666,"[{'alias': 'korean', 'title': 'Korean'}, {'ali...",4.5,"{'latitude': 34.065414, 'longitude': -118.3095...",[delivery],$$,"{'address1': '4163 W 5th St', 'address2': '', ...",+12133839499,(213) 383-9499,568.284344
3,qAAoilHU25Qr45FOQlA19g,kang-ho-dong-baekjeong-los-angeles-4,Kang Ho-dong Baekjeong,https://s3-media1.fl.yelpcdn.com/bphoto/0LQSEL...,False,https://www.yelp.com/biz/kang-ho-dong-baekjeon...,4902,"[{'alias': 'bbq', 'title': 'Barbeque'}, {'alia...",4.5,"{'latitude': 34.06375298900917, 'longitude': -...","[delivery, pickup]",$$$,"{'address1': '3465 W 6th St', 'address2': 'Ste...",+12133849678,(213) 384-9678,607.771480
4,iRjnWYY5oVJTaZ5teU1vOQ,king-chang-la-los-angeles,King Chang - LA,https://s3-media3.fl.yelpcdn.com/bphoto/FGJWXO...,False,https://www.yelp.com/biz/king-chang-la-los-ang...,283,"[{'alias': 'korean', 'title': 'Korean'}, {'ali...",5.0,"{'latitude': 34.06329, 'longitude': -118.30372}","[delivery, pickup, restaurant_reservation]",NaN,"{'address1': '3732 W 6th St', 'address2': None...",+12133185250,(213) 318-5250,14.161470


In [23]:
final_df.sort_values(by=['rating'], ascending=False)

,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,price,location,phone,display_phone,distance
199,EtYraN2nwiuv2WQR8ZHDQg,yang-ga-neh-los-angeles-2,Yang Ga Neh,https://s3-media2.fl.yelpcdn.com/bphoto/8yYZ4d...,False,https://www.yelp.com/biz/yang-ga-neh-los-angel...,11,"[{'alias': 'korean', 'title': 'Korean'}]",5.0,"{'latitude': 34.0485835075378, 'longitude': -1...",[],$,"{'address1': '1676 W 11th St', 'address2': '',...",+12133813358,(213) 381-3358,2967.565916
46,CBnXC267v0gjZVKWVWaNOA,gamsung-pocha-los-angeles,Gamsung Pocha,https://s3-media2.fl.yelpcdn.com/bphoto/f3aXTh...,False,https://www.yelp.com/biz/gamsung-pocha-los-ang...,14,"[{'alias': 'korean', 'title': 'Korean'}, {'ali...",5.0,"{'latitude': 34.05755, 'longitude': -118.29386}",[],NaN,"{'address1': '3074 W 8th St', 'address2': None...",+12139087843,(213) 908-7843,1137.018320
160,FZWyNLBS-2s9hlqp8chkzQ,ipds0901-los-angeles-2,Ipds0901,https://s3-media4.fl.yelpcdn.com/bphoto/EmSLgD...,False,https://www.yelp.com/biz/ipds0901-los-angeles-...,1,"[{'alias': 'korean', 'title': 'Korean'}]",5.0,"{'latitude': 34.05289, 'longitude': -118.27815}",[],NaN,"{'address1': '2010 James M Wood Blvd', 'addres...",,,2641.236423
154,d19eaXQrU2_ouSsaeARGeA,cho-dang-tofu-los-angeles,Cho Dang Tofu,https://s3-media1.fl.yelpcdn.com/bphoto/n8psf0...,False,https://www.yelp.com/biz/cho-dang-tofu-los-ang...,1,"[{'alias': 'korean', 'title': 'Korean'}, {'ali...",5.0,"{'latitude': 34.068589952986635, 'longitude': ...",[],NaN,"{'address1': '301 S Western Ave', 'address2': ...",+12133222562,(213) 322-2562,778.509039
29,LtQu-lT9Hpm_n_5LcXsdYA,sulga-house-of-beef-bone-soup-los-angeles,Sulga House of Beef Bone Soup,https://s3-media1.fl.yelpcdn.com/bphoto/VVqDqi...,False,https://www.yelp.com/biz/sulga-house-of-beef-b...,335,"[{'alias': 'korean', 'title': 'Korean'}, {'ali...",5.0,"{'latitude': 34.07642, 'longitude': -118.30557}","[delivery, pickup]",$$,"{'address1': '4451 Beverly Blvd', 'address2': ...",+13237410669,(323) 741-0669,1449.940570
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
158,hkiDk5vgVio0KZ3MI9zwaw,yet-gol-los-angeles,Yet Gol,https://s3-media3.fl.yelpcdn.com/bphoto/cyFwLW...,False,https://www.yelp.com/biz/yet-gol-los-angeles?a...,42,"[{'alias': 'korean', 'title': 'Korean'}]",3.5,"{'latitude': 34.0493392944336, 'longitude': -1...",[delivery],$,"{'address1': '1144 S Western Ave', 'address2':...",+13237328000,(323) 732-8000,1629.000847
198,4H_NG76ffZOuw79qmiRJxg,moobongri-soondae-무봉리-순대-본점-los-angeles,Moobongri Soondae 무봉리 순대 본점,https://s3-media1.fl.yelpcdn.com/bphoto/IdCPLo...,False,https://www.yelp.com/biz/moobongri-soondae-%EB...,106,"[{'alias': 'korean', 'title': 'Korean'}, {'ali...",3.5,"{'latitude': 34.0640652, 'longitude': -118.309...",[],$$,"{'address1': '545 S Western Ave', 'address2': ...",+12133871600,(213) 387-1600,538.050691
184,dWoQPRhhHPBAssKJy9MpBw,kyochon-los-angeles-3,KyoChon,https://s3-media2.fl.yelpcdn.com/bphoto/vMca-o...,False,https://www.yelp.com/biz/kyochon-los-angeles-3...,2534,"[{'alias': 'korean', 'title': 'Korean'}, {'ali...",3.5,"{'latitude': 34.06372, 'longitude': -118.30622}","[pickup, delivery]",$$,"{'address1': '3833 W 6th St', 'address2': '', ...",+12137399292,(213) 739-9292,241.449937
195,hjAAofyqVizqkjyGJ4Qr1A,namsan-los-angeles,Namsan,https://s3-media1.fl.yelpcdn.com/bphoto/HfwXXQ...,False,https://www.yelp.com/biz/namsan-los-angeles?ad...,255,"[{'alias': 'korean', 'title': 'Korean'}]",3.0,"{'latitude': 34.0637195, 'longitude': -118.301...","[pickup, delivery]",$$,"{'address1': '3613 W 6th St', 'address2': 'Ste...",+12133827777,(213) 382-7777,275.617752
